In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import numpy as np
import seaborn as sns

In [2]:
plt.rcParams["figure.figsize"]=[15,5]

In [3]:
raw_pp=pd.read_csv('pickandplace1.csv', usecols=['timestamp', 'data.ax', 'data.az'])
raw_pp.sort_values(by=['timestamp'], inplace=True)
raw_pp.reset_index(inplace=True)
raw_pp.dropna(axis=0, inplace=True)
raw_pp['timestamp'] =  pd.to_datetime(raw_pp['timestamp'])

In [4]:
raw_pp_current=pd.read_csv('pickandplace1.csv', usecols=['timestamp', 'data.A1', 'data.A2', 'data.A3'])
raw_pp_current.sort_values(by=['timestamp'])
raw_pp_current.reset_index(inplace=True)
raw_pp_current['total_current']=raw_pp_current['data.A1']+raw_pp_current['data.A2']+raw_pp_current['data.A3']
raw_pp_current['timestamp']=pd.to_datetime(raw_pp_current['timestamp'])
raw_pp_current.index=raw_pp_current['timestamp']

In [5]:
#Load LD data and plot
raw_pp["acc"] = ( raw_pp["data.ax"]**2 + raw_pp["data.az"]**2) ** 0.5
acc=raw_pp[["timestamp", "acc"]]
test_pp=acc
# test_pp['acc_rolling_sum']=pd.Series.to_frame(test_pp.acc.rolling(100*10, center=True).sum())
test_pp['acc_rolling_std']=pd.Series.to_frame(test_pp.acc.rolling(100*2, center=True).std())


/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
%matplotlib qt

In [ ]:
plt.plot(test_pp['timestamp'], test_pp['acc'])
plt.plot(test_pp['timestamp'], test_pp['acc_rolling_std'])

In [7]:
test_pp['detections']=pd.Series.to_frame(test_pp.acc_rolling_std.rolling(100, center=True).sum())

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
plt.plot(test_pp['timestamp'], test_pp['acc']*100)
plt.plot(test_pp['timestamp'], test_pp['detections'])

In [ ]:
# plt.plot(test_pp['timestamp'], test_pp['detections'])

In [8]:
test_pp['boards']=0
test_pp['boards'][test_pp['detections']>=2]=1

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
plt.plot(test_pp['timestamp'], test_pp['acc']*100)
plt.plot(test_pp["timestamp"],test_pp["boards"])

In [9]:
working_times_raw=scipy.signal.find_peaks(test_pp.boards, width=1)

In [10]:
working_times_raw_df=pd.DataFrame({"sample_number":working_times_raw[0], "working_time":working_times_raw[1]['widths']/72})

In [ ]:
test_pp['boards']=test_pp['boards']*-1
idle_times_raw=scipy.signal.find_peaks(test_pp.boards, width=1)
idle_times_raw_df=pd.DataFrame({"sample_number":idle_times_raw[0], "working_time":idle_times_raw[1]['widths']/72})
test_pp['boards']=test_pp['boards']*-1

In [ ]:
for x, row in working_times_raw_df.iterrows():
    working_times_raw_df.ix[x,'start_time']=test_pp.iloc[int(working_times_raw[1]['left_ips'][x])].timestamp
    working_times_raw_df.ix[x,'end_time']=test_pp.iloc[int(working_times_raw[1]['right_ips'][x])].timestamp
    working_times_raw_df.ix[x,'timestamp']=test_pp.iloc[int(working_times_raw[0][x])].timestamp

In [ ]:

for x, row in idle_times_raw_df.iterrows():
    idle_times_raw_df.ix[x,'start_time']=test_pp.iloc[int(idle_times_raw[1]['left_ips'][x])].timestamp
    idle_times_raw_df.ix[x,'end_time']=test_pp.iloc[int(idle_times_raw[1]['right_ips'][x])].timestamp
    idle_times_raw_df.ix[x,'timestamp']=test_pp.iloc[int(idle_times_raw[0][x])].timestamp

In [ ]:
working_times_df=working_times_raw_df[working_times_raw_df.working_time>10]

In [ ]:
working_times_df['event']=1
idle_times_raw_df['event']=0

In [ ]:
# working_times_df['energy']=raw_pp_current[(raw_pp_current['timestamp'] <= working_times_df['end_time']) & (raw_pp_current['timestamp'] >= working_times_df['start_time'])]['total_current'].sum()*230

In [ ]:
working_times_df['energy'] = working_times_df.apply(lambda x: raw_pp_current.loc[(raw_pp_current.timestamp <= x.end_time) & 
                                                            (x.start_time <= raw_pp_current.timestamp),
                                                            ['total_current']].sum()*230/3600000, axis=1)

In [ ]:
# for index, x in enumerate(working_times_raw[0]):
#     working_times_raw_df.ix[index,'energy']=np.trapz(raw_pp_current.ix[int(working_times_raw[1]['left_ips'][index]):int(working_times_raw[1]['right_ips'][index]), 'data.A1'].tolist())
# working_times_raw_df['energy']=working_times_raw_df['energy']*230/3600000

In [ ]:
idle_times_raw_df['energy'] = idle_times_raw_df.apply(lambda x: raw_pp_current.loc[(raw_pp_current.timestamp <= x.end_time) & 
                                                            (x.start_time <= raw_pp_current.timestamp),
                                                            ['total_current']].sum()*230/3600000, axis=1)

In [ ]:
# for index, x in enumerate(idle_times_raw[0]):
#     idle_times_raw_df.ix[index,'energy']=np.trapz(raw_pp_current.ix[int(idle_times_raw[1]['left_ips'][index]):int(idle_times_raw[1]['right_ips'][index]), 'data.A1'].tolist(), dx=1)
# idle_times_raw_df['energy']=idle_times_raw_df['energy']*230/3600000

In [ ]:
working_times_df.index=working_times_df.timestamp
working_times_df.drop('timestamp', axis=1, inplace=True)
working_times_df.drop('sample_number', axis=1, inplace=True)

In [ ]:
idle_times_raw_df.index=idle_times_raw_df.timestamp
idle_times_raw_df.drop('timestamp', axis=1, inplace=True)
idle_times_raw_df.drop('sample_number', axis=1, inplace=True)

In [ ]:
# plt.plot(test_pp["timestamp"],test_pp["boards"])
# plt.plot(raw_pp_current.timestamp, raw_pp_current['total_current'])

In [ ]:
PP_events=working_times_df[['event', 'working_time', 'energy']]

In [ ]:
idle_times_df=idle_times_raw_df[['event', 'working_time', 'energy']]
PP_events=PP_events.append(idle_times_df)

In [ ]:
# sns.distplot(PP_events[PP_events.event==1].working_time)
# plt.title('PP1 Working Times Histogram')

In [ ]:
# print('PP working time mode: ', PP_events[PP_events.event==1].working_time.mode().mean())

In [ ]:
# sum(raw_pp_current.total_current.tolist())

In [ ]:
# PP_events.loc[PP_events.event==0, 'energy']

In [ ]:
plt.plot(test_pp.timestamp, test_pp.acc)

In [ ]:
PP_events.sort_values(by=['timestamp'], inplace=True)

In [ ]:
# plt.plot(PP_events.index, PP_events.event)
plt.plot(test_pp["timestamp"],test_pp["boards"])

In [ ]:
# PP_events[PP_events.event==0]

In [ ]:
plt.plot(raw_pp_current.timestamp,raw_pp_current.total_current)

In [ ]:
plt.stem(idle_times_df.index, idle_times_df.event)

In [ ]:
# sns.distplot(idle_times_df.power)

In [ ]:
working_times_df['power']=working_times_df['energy']*3600000/working_times_df['working_time']

In [ ]:
#sns.distplot(PP_events[PP_events.event==1].working_time, bins=50)

In [ ]:
PPdist=PP_events[PP_events.event==1]

In [ ]:
PPdist

In [ ]:
PPdist=PPdist[PPdist.working_time < np.percentile(PPdist.working_time,95)]

In [ ]:
print(PPdist.working_time.max())
print(PPdist.working_time.min())

In [ ]:
binnum=int(round((PPdist.working_time.max()-PPdist.working_time.min())/5))
binnum

In [ ]:
sns.distplot(PP_events[PP_events.event==1].working_time, bins=binnum)

In [ ]:
modeWT=pd.cut(PPdist[PPdist.event==1]["working_time"], bins=binnum).value_counts().idxmax().mid

In [ ]:
modeWT

In [ ]:
# statistics.mode(PP_events[PP_events.event==1].working_time.tolist(), bins=10)

In [ ]:
plt.plot(test_pp.timestamp, test_pp.acc)
plt.plot(test_pp.timestamp, test_pp.boards)